4. Canalització
- Which kind of actors are the leaders of the project? (com es canalitzen els recursos, per tipus d’actor?)
- Which typology of projects are doing the actors? (com els canalitzen els diferents agents, per tipologies?)

#### Steps in Sankey

GovLevel -> TupuActor -> Countries (Pais)

#### json structure

`[2015, 2016, 2017, 2018]`

each year

`2015:{nodes:[{...}], links:[{...}]}`

In [1]:
import pandas as pd
import numpy as np
import json

pd.options.display.max_columns = 2000 # remove ellipsis
pd.options.display.max_rows = 2000 # remove ellipsis

In [24]:
df = pd.read_csv('data/DataPross+Pop+Budget.csv',encoding='latin1', decimal=",")
#df_new = pd.read_excel('data/dataset cooperacio nomes Generalitat 2015-2018.xlsx')

In [25]:
df.head()

,Index,Modalitat,Any,GovLevel,Institucio,InstitucioII,Codi,Entitat,Subvenció,Budget,Population,CRS,Projecte,Pais,TipusActor,Subvenció(TXT),Area,Ciutat,ConcepteFCCD,Contrapart,SectorCAD,ODS,DretsHumans,CulturadelaPau,Sostenibilitat,EquitatdeGenere.,Sensibilitzacio,Formacio,Recerca,Mobilitzacio,PaisCat,IDH,Lat,long
0,1.0,Desenvolupament,2016.0,Municipal,AjBarcelona,Barcelona,80193.0,Farmaceutics Mundi,143398.12,24687931401,1636762.0,15170 - Organitzacions i institucions per la i...,Apoyo a restitución integral de los DD de las ...,Democratic Republic of the Congo,NaN,"143.398,12 ",NaN,Kivu,NaN,FEPSI (Femmes Engagées pour la Promotion de la...,151.0,5.0,x,x,x,x,NaN,NaN,NaN,NaN,República Democràtica del Congo,0.43,NaN,NaN
1,3.0,Acció_Humanitària,2016.0,Municipal,AjBarcelona,Barcelona,80193.0,Acció contra la Fam,153500.00,24687931401,1636762.0,73010 - Ajuda per a la reconstrucció,Capacidades institucionales y comunitarias par...,Palestine,NaN,"153.500,00 ",NaN,GAZA,NaN,Action Against Hunger - Occupied Palestinian T...,740.0,15.0,x,NaN,x,x,NaN,NaN,NaN,NaN,Palestina,0.68,NaN,NaN
2,4.0,Desenvolupament,2016.0,Municipal,AjBarcelona,Barcelona,80193.0,Fundació Montblanc per a la promoció de la dona,57438.68,24687931401,1636762.0,"13040 - Lluita contra ETS, VIH inclosa",PREVENCIÓ I LLUITA CONTRA LA SIDA A 2 DISTRICT...,Democratic Republic of the Congo,NaN,"57.438,68 ",NaN,Kinshasa City,NaN,"ASSOCIATION CENTRE CONGOLAIS DE CULTURE, DE FO...",130.0,5.0,x,x,x,x,NaN,NaN,NaN,NaN,República Democràtica del Congo,0.43,NaN,NaN
3,5.0,Desenvolupament,2016.0,Municipal,AjBarcelona,Barcelona,80193.0,Metges del Món,47440.00,24687931401,1636762.0,15160 - Drets Humans,LGTBI - Dakar,Senegal,NaN,"47.440,00 ",NaN,Dakar,NaN,"Association KIRAAY, Association ADAMA, Associa...",151.0,5.0,x,NaN,NaN,x,NaN,NaN,NaN,NaN,Senegal,0.47,NaN,NaN
4,6.0,Acció_Humanitària,2016.0,Municipal,AjBarcelona,Barcelona,80193.0,Acció contra la Fam,57440.00,24687931401,1636762.0,73010 - Ajuda per a la reconstrucció,Medios de vida productivos para mujeres cabeza...,Palestine,NaN,"57.440,00 ",NaN,CISJORDANIA,NaN,Action Against Hunger - Occupied Palestinian T...,740.0,15.0,x,x,NaN,x,NaN,NaN,NaN,NaN,Palestina,0.68,NaN,NaN


In [26]:
df2 = df[['Any', 'GovLevel', 'Entitat', 'Pais', 'Subvenció']].dropna(how='all')

In [28]:
df2['Entitat'] = df2['Entitat'].fillna('Unknown')

In [38]:
def json_maker(year): 
    df3 = df2[df2['Any']==year]
    table = pd.pivot_table(df3, values=['Subvenció'], index=['GovLevel', 'Entitat'],
                    aggfunc={'Subvenció': np.sum}).reset_index()
    table2= pd.pivot_table(df3, values=['Subvenció'], index=['Entitat', 'Pais'],
                    aggfunc={'Subvenció': np.sum}).reset_index()
    
    t1 = table.rename(columns={"GovLevel": "target", 
                               "Entitat": "source", 
                               "Subvenció": "value"})
    t2 = table2.rename(columns={"Entitat": "target", 
                                "Pais": "source", 
                                "Subvenció": "value"})
    
    links = pd.concat([t1, t2], ignore_index=True)
    result = links.to_json(orient="records")
    parsed_links = json.loads(result)
    
    
    nodes = list(links2018['target'].unique()) + list(links2018['source'].unique())
    nodes_list = list(set(nodes))    
    nodes_df = pd.DataFrame(nodes_list, columns = ["id"])
    result = nodes_df.to_json(orient="records")
    parsed_nodes = json.loads(result)

    d = { "nodes": parsed_nodes, "links": parsed_links }
    
    return d 

In [50]:
data = {"2015":json_maker(2015), "2016":json_maker(2016), "2017":json_maker(2017), "2018":json_maker(2018)}

In [53]:
data.keys()

dict_keys(['2015', '2016', '2017', '2018'])

In [55]:
data['2015']

{'nodes': [{'id': 'Porqueres'},
  {'id': 'Miquel Carrillo Ponce'},
  {'id': 'Associació Amics de Puerto Cabezas'},
  {'id': 'Uganda'},
  {'id': 'Càritas'},
  {'id': 'Mali'},
  {'id': 'Sant Boi de Llobregat'},
  {'id': 'Vidreres'},
  {'id': 'Brazil'},
  {'id': "Sant Sadurní d'Anoia"},
  {'id': 'gambia'},
  {'id': 'Vacarisses'},
  {'id': 'Santa Coloma de Gramenet'},
  {'id': 'Castellar del Vallès'},
  {'id': 'Sentmenat'},
  {'id': 'Alcanar'},
  {'id': 'Fundació Grup Tercer Món \x96 Mataró'},
  {'id': 'Consell Comarcal del Maresme'},
  {'id': 'Tiana'},
  {'id': 'Fontcoberta'},
  {'id': 'ACOES Catalunya'},
  {'id': 'Ajuntament de Molins de Reis'},
  {'id': 'Castelldefels'},
  {'id': 'Girona'},
  {'id': 'United Republic of Tanzania'},
  {'id': 'Caldes de Malavella'},
  {'id': 'Fundación Daniel Shah & Núria Toneu'},
  {'id': 'Associació Molins de Rei amb el Sàhara'},
  {'id': 'Sant Vicenç de Montalt'},
  {'id': 'San-Prem'},
  {'id': 'America Central i Carib'},
  {'id': 'Olot'},
  {'id': 'Equ

In [54]:
with open("output/data.json", "w") as outfile: 
    json.dump(data, outfile)

In [32]:
df2018 = df2[df2['Any']==2018]

### InstitucioII -> Entitat

In [33]:
table = pd.pivot_table(df2018, values=['Subvenció'], index=['InstitucioII', 'Entitat'],
                    aggfunc={'Subvenció': np.sum}).reset_index()

KeyError: 'InstitucioII'

In [13]:
table2= pd.pivot_table(df2018, values=['Subvenció'], index=['Entitat', 'Pais'],
                    aggfunc={'Subvenció': np.sum}).reset_index()

In [14]:
t1 = table.rename(columns={"InstitucioII": "target", "Entitat": "source", "Subvenció": "value"})

In [15]:
t2 = table2.rename(columns={"Entitat": "target", "Pais": "source", "Subvenció": "value"})

In [16]:
links2018 = pd.concat([t1, t2], ignore_index=True)

In [17]:
nodes = list(links2018['target'].unique()) + list(links2018['source'].unique())
nodes_list = list(set(nodes))

In [18]:
nodes2018 = pd.DataFrame(nodes_list, columns = ["id"])

In [22]:
links2018.head(10)

,target,source,value
0,Abrera,Unknown,1550.0
1,Aiguafreda,Unknown,2425.0
2,Aiguamúrcia,Unknown,200.0
3,Alcanar,Unknown,875.0
4,Alella,Col·lectiu Maloka Colombia,6000.0
5,Alella,Creu Roja,5500.0
6,Alella,Fundació privada el maresme pro persones amb d...,6000.0
7,Alella,Medicus Mundi Mediterrània,6500.0
8,Alella,Solidança,7000.0
9,Alella,Unknown,23025.0


In [21]:
nodes2018.head()

,id
0,Porqueres
1,Miquel Carrillo Ponce
2,Associació Amics de Puerto Cabezas
3,Uganda
4,Càritas


In [140]:
result = nodes2018.to_json(orient="records")
parsed_nodes = json.loads(result)

In [178]:
parsed_nodes

[{'id': 'Consell Comarcal del Maresme'},
 {'id': 'Caldes Solidaria'},
 {'id': 'Cardedeu'},
 {'id': 'Navarcles'},
 {'id': 'Martorelles'},
 {'id': 'Fundació Josep Comaposada \x96 sindicalistes solidaris'},
 {'id': 'Tortosa'},
 {'id': 'Les'},
 {'id': 'Fundació Ètnia'},
 {'id': "Santa Cristina d'Aro"},
 {'id': 'Montornès del Vallès'},
 {'id': 'El Salvador'},
 {'id': 'Tiana'},
 {'id': 'Montbrió del Camp'},
 {'id': 'CCASPS \x96 Coor. Catalana d\x92ajuntaments solidaris amb el poble saharauí'},
 {'id': 'Tossa de Mar'},
 {'id': 'Fundación Aito García Reneses'},
 {'id': 'Santa Perpètua de Mogoda'},
 {'id': 'Canyelles'},
 {'id': 'Castelldefels'},
 {'id': 'Sallent'},
 {'id': 'Barcelona'},
 {'id': 'el Prat de Llobregat'},
 {'id': 'Xarxa de Consum Solidari'},
 {'id': 'Diputació de Tarragona'},
 {'id': 'Fundació privada el maresme pro persones amb disminució psíquica'},
 {'id': 'Sant Fost de Campsentelles'},
 {'id': 'Montcada i Reixac'},
 {'id': 'Creixell'},
 {'id': 'Palamós'},
 {'id': 'Fundació Pri

In [141]:
result2018 = links2018.to_json(orient="records")
parsed_links = json.loads(result2018)

In [185]:
d = { "nodes": parsed_nodes, "links": parsed_links }

In [186]:
with open("output/data2018.json", "w") as outfile: 
    json.dump(d, outfile)

In [158]:
## create a sample

In [163]:
s_list = t1[t1['source'] != "Unknown"][:20]['source'].to_list()

In [168]:
t1_sample = t1[t1['source'] != "Unknown"][:20]

In [167]:
t2_sample = t2[t2['target'].isin(s_list)]

In [169]:
links2018_s = pd.concat([t1_sample, t2_sample], ignore_index=True)

In [170]:
nodes_s = list(links2018_s['target'].unique()) + list(links2018_s['source'].unique())
nodes_list_s = list(set(nodes_s))

In [171]:
nodes2018_s = pd.DataFrame(nodes_list_s, columns = ["id"])

In [172]:
result_s = nodes2018_s.to_json(orient="records")
parsed_nodes_s = json.loads(result_s)

In [173]:
result2018_s = links2018_s.to_json(orient="records")
parsed_links_s = json.loads(result2018_s)

In [182]:
sample = { "nodes": parsed_nodes_s, "links": parsed_links_s }

In [183]:
with open("output/data2018_sample.json", "w") as outfile: 
    json.dump(sample, outfile)

In [184]:
parsed_nodes_s

[{'id': 'Honduras'},
 {'id': 'Col·lectiu Maloka Colombia'},
 {'id': 'Ulls del Món'},
 {'id': 'Cabrera de Mar'},
 {'id': 'Fundació privada el maresme pro persones amb disminució psíquica'},
 {'id': 'Catalunya'},
 {'id': 'Fundació Wassu'},
 {'id': 'Acció Solidària i Logística - ASL'},
 {'id': 'Senegal'},
 {'id': 'Medicus Mundi Mediterrània'},
 {'id': 'Begur'},
 {'id': 'Creu Roja'},
 {'id': 'Bolivia'},
 {'id': 'Peru'},
 {'id': 'Asociación Proa (Pro-Activa Open Arms)'},
 {'id': 'GERDD \x96 Gestora de recursos per al desenvolupament del discapacitat'},
 {'id': 'Alella'},
 {'id': 'OUDIODIAL'},
 {'id': 'Associació d\x92amics del bisbe Joan Godayol'},
 {'id': 'Arbúcies'},
 {'id': 'Mediterrania'},
 {'id': 'Guatemala'},
 {'id': 'Coordinadora d\x92Associacions Senegaleses de Catalunya CASC'},
 {'id': 'Barberà del Vallès'},
 {'id': 'El Salvador'},
 {'id': 'Mozambique'},
 {'id': 'Colombia'},
 {'id': 'Comissió Agenda Llatinoamericana'},
 {'id': 'CCASPS \x96 Coor. Catalana d\x92ajuntaments solidaris 

In [23]:
sample

NameError: name 'sample' is not defined